Links to files and data previews were removed so to respect company's data privacy

# Data Science Homework

Your homework, should you choose to accept it, will be to design models for predicting sales of used cars in the USA.  The data for this task is contained in the following files:

•	Link 1

•	Link 2

(If you prefer working with SQL, the data is also available here .)

listings_sample.csv contains vehicles that were listed for sale by some car dealerships, with one row for each day that the vehicle was listed for sale.  sales_sample.csv contains the date of sale of each vehicle.   Vehicles are uniquely identified by their listing_id.

There are six tasks described below. Please complete each of the six tasks in language(s) of your choosing.  You should submit the source code along with the requested output files and instructions on how to execute the code to reproduce your results.  Place all the requested files in a directory data_science_homework, and submit a zipped or otherwise compressed file.  


In [1]:
import pandas as pd
import numpy as np


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
sales = pd.read_csv('sales_sample.csv', sep = ',')
listings = pd.read_csv('listings_sample.csv',  sep = ',', low_memory=False)

In [3]:
print(sales.shape)
print(sales.columns)


(19463, 3)
Index(['date_sold', 'listing_id', 'days_on_market'], dtype='object')


In [27]:
sales.head(0)

,date_sold,listing_id,days_on_market


In [5]:
sales.describe()

,listing_id,days_on_market
count,1.946300e+04,19161.000000
mean,2.976610e+16,46.497678
std,5.318423e+18,56.203556
min,-9.220673e+18,0.000000
25%,-4.552788e+18,11.000000
50%,3.431005e+16,28.000000
75%,4.629118e+18,62.000000
max,9.223240e+18,1039.000000


In [6]:
print(listings.shape)
print(listings.columns)


(869049, 22)
Index(['date_added', 'has_accidents', 'body_type_name', 'car_year', 'listing_id', 'days_on_market', 'exterior_color', 'highway_fuel_economy', 'doors', 'make_display_name', 'model_display_name', 'mileage', 'msrp', 'color', 'owner_count', 'price', 'estimated_discount', 'seller_city', 'seller_postal_code', 'seller_id', 'seller_rating', 'trim_name'], dtype='object')


In [28]:
listings.head(0)

,date_added,has_accidents,body_type_name,car_year,listing_id,days_on_market,exterior_color,highway_fuel_economy,doors,make_display_name,model_display_name,mileage,msrp,color,owner_count,price,estimated_discount,seller_city,seller_postal_code,seller_id,seller_rating,trim_name,date_added_date


In [8]:
listings.describe()

,car_year,listing_id,days_on_market,highway_fuel_economy,mileage,msrp,owner_count,price,estimated_discount,seller_postal_code,seller_id,seller_rating
count,869049.000000,8.690490e+05,859434.000000,584394.000000,861301.000000,534388.000000,719957.000000,822981.000000,869049.000000,869049.000000,8.690490e+05,810782.000000
mean,2015.082083,6.173419e+15,55.813157,22.449873,65073.773858,20008.460502,1.548237,29779.875199,1347.291391,98503.301443,-2.739658e+17,4.309527
std,4.678398,5.312801e+18,75.426711,3.209529,53195.063530,21694.741044,0.995236,15556.300444,1402.287093,521.297155,5.151930e+18,0.607909
min,1978.000000,-9.220673e+18,0.000000,12.000000,50.000000,-1.000000,1.000000,-1.000000,0.000000,37122.000000,-8.955974e+18,1.000000
25%,2013.000000,-4.578290e+18,13.000000,21.000000,23510.000000,-1.000000,1.000000,18995.000000,229.000000,98166.000000,-4.585827e+18,4.000000
50%,2017.000000,1.476116e+16,34.000000,22.000000,49159.000000,17358.000000,1.000000,28999.000000,1004.000000,98272.000000,4.617962e+17,4.000000
75%,2019.000000,4.573404e+18,72.000000,24.000000,93685.000000,37635.000000,2.000000,38910.000000,2007.000000,98903.000000,4.210657e+18,5.000000
max,2021.000000,9.223240e+18,1784.000000,37.000000,378779.000000,435125.000000,14.000000,538531.000000,25127.000000,99515.000000,9.191927e+18,5.000000


#### Let's check for duplications

In [9]:
listings['date_added_date'] = pd.to_datetime(listings['date_added']).dt.date
listings.listing_id.nunique()
listings[listings.duplicated(subset=['listing_id', 'date_added_date'], keep = False)][listings.listing_id \
                                > 0].sort_values(by = ['listing_id', 'date_added', 'days_on_market']).head(0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,date_added,has_accidents,body_type_name,car_year,listing_id,days_on_market,exterior_color,highway_fuel_economy,doors,make_display_name,model_display_name,mileage,msrp,color,owner_count,price,estimated_discount,seller_city,seller_postal_code,seller_id,seller_rating,trim_name,date_added_date


In [10]:
print("unique listing id to to all listing id ratio:", sales.listing_id.nunique() / sales.listing_id.shape[0])

print("duplicated sales:", sales[sales.duplicated(subset=['listing_id'], keep = False)].shape[0])

sales[sales.duplicated(subset=['listing_id'], keep = False)].sort_values(by = 'listing_id').head(0)

unique listing id to to all listing id ratio: 0.9908544417612907
duplicated sales: 330


,date_sold,listing_id,days_on_market


### Interpretation

We can see that both: some sales and listings got duplicated. The listings are duplicated when initially the data is entered not full: and later some additional information is added

I do not have information why these listing ids are duplicated in the sales dataset.

I could assume several things: 
    
1) some of these listing ids are negative numbers which doesn't make much sense - these could be faulty entries

3) Duplicated because some listings are dulicated - wrong since sales dates differ even for duplicated listing_ids

2) Some listings are duplicated because I see reservations not actual sales and these reservations due to some reason were canceled

#### I will assume the 3 - I will hence leave only the very last sale for every listing_id.  I will also keep only the correct listings.

In [11]:
print(sales.shape)
sales = sales.sort_values(by = ['listing_id', 'date_sold']).drop_duplicates(['listing_id'], keep='last')
print(sales.shape)

(19463, 3)
(19285, 3)


In [12]:
print(listings.shape)
listings = listings.sort_values(by = ['listing_id', 'date_added_date', \
                                      'days_on_market']).drop_duplicates(['listing_id', 'date_added_date'], keep='first')

print(listings.shape)

(869049, 23)
(861584, 23)


#### Now we can merge the listings dataset with sales dataset

In [13]:
print(listings.shape)
df = listings.merge(sales, how='left', on=['listing_id'])

# Remove time of sales and leave only dates
print(df.shape)
df['date_added'] = pd.to_datetime(df['date_added']).dt.date
df['date_sold'] = pd.to_datetime(df['date_sold']).dt.date

# Flag for days when the listed item was sold
df['sold'] = np.where(df['date_added'] == df['date_sold'], 1, 0)


df.head(0)

(861584, 23)
(861584, 25)


,date_added,has_accidents,body_type_name,car_year,listing_id,days_on_market_x,exterior_color,highway_fuel_economy,doors,make_display_name,model_display_name,mileage,msrp,color,owner_count,price,estimated_discount,seller_city,seller_postal_code,seller_id,seller_rating,trim_name,date_added_date,date_sold,days_on_market_y,sold


listings[listings.listing_id == -8936203189521311158].head().sort_values(by = 'date_added')

### Task 1

Produce a CSV report of the number of vehicles sold each day by each dealer for each valid pair of make and model values.  

The CSV report should have five columns - date, seller_id, make, model, and sales_count - and should be in a file titled make_model_sales_per_dealer_per_day.csv.  

The report should include days when no vehicles were sold, in which case sales_count = 0.  

Limit the CSV file to the first 100,000 rows sorted by seller_id, date, make, and then model.  

Some relevant information for this task: 

a)	The dealer is identified by the seller_id of the listings. 

b)	The make and model are identified by make_display_name and model_display_name. 

c)	Sold vehicles are those which are present in sales_sample.csv, which contains the date of sale of the vehicle.


In [14]:
df.sort_values(by = ['listing_id', 'date_added']).head(0)

,date_added,has_accidents,body_type_name,car_year,listing_id,days_on_market_x,exterior_color,highway_fuel_economy,doors,make_display_name,model_display_name,mileage,msrp,color,owner_count,price,estimated_discount,seller_city,seller_postal_code,seller_id,seller_rating,trim_name,date_added_date,date_sold,days_on_market_y,sold


In [15]:
# Grouping
grouped = df[['date_added','seller_id', 'make_display_name', 'model_display_name', 'sold']].groupby(['date_added',
            'seller_id', 'make_display_name', 'model_display_name'])[['sold']].sum().reset_index()

#Renamingcolumns and sorting
grouped = grouped.rename(columns={'date_added':'date', 'make_display_name':'make', 
                'model_display_name':'model'}).reset_index(drop = True).sort_values(by = ['seller_id', 'date', 'make', 'model'])

# Let's write the output to a csv file
grouped.head(100000).to_csv('make_model_sales_per_dealer_per_day.csv', sep=',',index=False)

grouped.head(0)

,date,seller_id,make,model,sold


In [16]:
# Sanity check.
#Check if sum of sales in the output file is the same as in the sales file (after removal of duplicates)

sales.shape[0] == sum(grouped.sold)

True

### Task 2

Produce a csv file of the number of vehicles sold in the preceding 30 days, excluding the current day, for each day, dealer, make, and model.  

As with task 1, the counts should be per dealer, make, and model, and should include days when no vehicles were sold in the preceding 30 days.  

The file should be titled make_model_sales_per_dealer_previous_30_days.csv and have columns date, seller_id, make, model, and sales_past_30days_per_make_model_dealer.  

Limit the CSV file to the first 100,000 rows sorted by seller_id, date, make, and then model.

In [17]:
# Let's interpolate the missing dates and include 0 in the sales between dates in listings 
# as later I will use the "rolling" function which doesn't handle missing values

grouped['date']  = pd.to_datetime(grouped.date).astype('datetime64[ns]')

f = lambda x: x.reindex(pd.date_range(x.index.min(), x.index.max(), freq='D', name='date'))

grouped = grouped.set_index('date').groupby(['seller_id', 'make', \
                'model']).apply(f).drop(['seller_id', 'make', 'model'], axis=1).fillna(0).sort_values(by = ['seller_id', 'date']).reset_index()


In [18]:
#  Let's calculate vehicles sold in the preceding 30 days
rolling = grouped.set_index('date').groupby(['seller_id', 'make', 'model']).rolling(window=30, freq='D', \
                                        min_periods=1).sum()['sold'].shift().reset_index().rename(columns={  \
                                        'sold':'sales_past_30days_per_make_model_dealer'}).fillna(0)


In [19]:
print(rolling.shape)
print(grouped.shape)
rolling = rolling.merge(grouped, on = ['seller_id', 'date', 'make', 'model'], how = \
                        'inner').sort_values(by = ['seller_id', 'date', 'make', 'model'])
print(rolling.shape)

# Let's write the output to a csv file
rolling[['date', 'seller_id', 'make', 'model', 'sales_past_30days_per_make_model_dealer']].head(100000).to_csv('make_model_sales_per_dealer_previous_30_days.csv', sep=',',index=False)

(621805, 5)
(621805, 5)
(621805, 6)


In [20]:
# Sanity check.

sales.shape[0] == sum(rolling.sold)

True

In [21]:
# Chenged the ordering to check on the same seller and model level over consecutive days - it's correct

rolling.sort_values(by = ['seller_id','make', 'model', 'date']).head(20)

,seller_id,make,model,date,sales_past_30days_per_make_model_dealer,sold
0,-8955973641505802610,Cadillac,Escalade,2020-02-15,0.0,0.0
1,-8955973641505802610,Cadillac,Escalade,2020-02-16,0.0,0.0
2,-8955973641505802610,Cadillac,Escalade,2020-02-17,0.0,0.0
3,-8955973641505802610,Cadillac,Escalade,2020-02-18,0.0,0.0
4,-8955973641505802610,Cadillac,Escalade,2020-02-19,0.0,0.0
5,-8955973641505802610,Cadillac,Escalade,2020-02-20,0.0,0.0
6,-8955973641505802610,Cadillac,Escalade,2020-02-21,0.0,0.0
7,-8955973641505802610,Cadillac,Escalade,2020-02-22,0.0,1.0
8,-8955973641505802610,Cadillac,Escalade,2020-02-23,1.0,0.0
9,-8955973641505802610,Cadillac,Escalade,2020-02-24,1.0,0.0


### Task 3

Calculate Pearson’s correlation coefficient between make and models sold by a dealer in the previous 30 days with the number sold for the same dealer, make and model in the next 30 days.

Write the result to a csv file correlation.csv with one row and one column called correlation.

In [22]:
rolling_forward = grouped.set_index('date').groupby(['seller_id', 'make', \
                'model']).rolling(window=30, freq='D',  min_periods=1).sum()['sold'].shift(-30).reset_index().rename(columns={'sold':'sold_rolling_forward_30'})

rolling_forward = rolling_forward.merge(grouped, on = ['seller_id', 'date', 'make', 'model'], how = 'inner')
print(rolling_forward.shape)


(621805, 6)


In [23]:
rolling = rolling.merge(rolling_forward, on = ['seller_id', 'date', 'make', 'model', 'sold'], how = 'inner')
rolling.shape

(621805, 7)

In [24]:
corr = rolling['sales_past_30days_per_make_model_dealer'].corr(rolling['sold_rolling_forward_30'], method='pearson')
print("correlation is: ",  corr)
pd.DataFrame([corr], columns=['Correlation']).to_csv('correlation.csv', sep='\t',index=False)

correlation is:  0.6355040612047936


### The next tasks require in depth dive into data/modeling and thus I will only answer them in writing, in particular, how I would try to approach the problems.

### Task 4

Suppose that we wish to predict the value for the next 30 days of sales for each combination of dealer, make, model. Describe the steps you would take to build a model for this. Code is not required for this, but feel free to include any data or graphs that help illustrate your reasoning. 

Put your response in a format of your choosing in a subdirectory 

a)	Which model or models you would try first, and why.

b)	Additional features you would consider developing, and why.

c)	Any specific problems you might foresee arising for this particular modeling task, and what you might do to handle them. 


### Answer

We can see there is some moderate correlation (0.64) between past and future 30 day sales. I would add additionally longer dataframes of sales for every seller: 30,60,...365 days sales for existing combinations of cars and makesa as a feature to predict the future ones. 

Also maybe there is a connection days_on_market and sales (i.e. potential discounts if the car hasn't been bought for a while). Car popularity metrics - i.e. average time body type/model is listed (you can add more features combining models with some other aspects - like has damage etc). one hot encoding of number car make/models (or their groups) that are listed for each seller - maybe the seller specialises in sales of cars with damage for example.

Car specific values like min/mean/max mileage, car_year, price, color, fraction of cars with has_accidents should also be useful when predicting if they will be sold into each seller's cars on display. The more data you have the more specific you can get, feature engineering would be the most important step. Also the data should then be scaled before using it in regressions, especially NN.

Given that the dataset is fairly large, NN models could work, for example LSTM, or 1 dimensional CNN models generally are known to perform well on multivariate time series problems. We should then create a supervised learning problem from the dataset, combining every seller at every point in time where sales 30 days in advance are known and then we could then fit the model, later testing on unseen data.

### Task 5

Suppose that we wish to model the probability of each individual listing selling in the next 7 days.  Describe how you might approach building a model for this.  Code is not required for this, but feel free to include any data or graphs that help illustrate your reasoning.  Put your response in a format of your choosing in a subdirectory data_science_homework/sales_probability.  Please do include the following information:

a)	Which model or models you would try first, and why.

b)	Additional features you would consider developing, and why.

c)	Any specific problems you might foresee arising for this particular modeling task, and what you might do to handle them. 


### Answer

This would be a classical classification task. You define a target for every listing as either sold or not during the next 7 days. I would test multiple models including xgboost, random forest, svm, logistic regression to see which one fits best (I prefer to use confusion matrices and AUROC to see how well they fit, since they are insensitive to class imbalance). The models return an estimation of "probability" of sales for the target, i.e. whether a sale will happen in the next 7 days. 

Here I believe listed-car specific features would be most important. Feature discretisation (on year, price, mileage, car tye (SUV, family car, etc) could help to make the model more robust to overfitting. One hot encoding of color vaiable would be needed and the least popular colors could be listed as 'other' to reduce dimmensionality.

One consideration would be whether to include same listing over and over for every date when it's iisted or only once/twice - when listing was sold/noxt sold in the next 7 days. I would do the former and include every listing for every listed day in both training and testing datasets - larger dataset and higher variation of features; however, the cars listing_ids in training and testing datasets would differ to avoid possible info leakage.

Additionally, there is some class imbalance - the number of listings is >40 times higher than the number of sales. However, the class imbalance would decrease if we mark all listings <7 days before sale as "sold". Nevertheless, to deal with class imbalance problem, undersampling/oversampling (SMOTE is quite good) could be helpful. It should be tested if oversamling outperforms original dataset with simply model hyperparameters adjusted for class imbalance - it can be either way from my experience.

### Task 6

Describe any additional observations and/or thoughts you had about this dataset in a subdirectory called data_science_homework/additional_observations, in a file with a format of your choosing.

### Thoughts/observations
__1)__. 
There were duplicates/some incomplete entries in sales and listings dataset - that I covered above, before Task 1.


__2)__ I did not dive into other variables since the tasks didn't reaquire me to.


__3)__ I hope I interpreted the 1) task correctly. These parts were a bit confusing:

"<...> for each valid pair of make and model values <...> The report should include days when no vehicles were sold, in which case sales_count = 0."

Initially I added all of the existing variations of every model and make for every seller. Also I used reindexing to add every missing date between the missing dates of listings. However, then the dataset got really huge and I thought that probably the task was much simpler and wanted me to only include: 

__a)__ for each valid pair of make and model values __that the seller had listed__.

__b)__ no interpolation of dates was needed but instead I simply had to keep the listings without sales in the dataset

So I did that.